This project examines mean reversion in U.S. equity returns and the predictive power of dividend yield for returns and dividend growth using a long-term dataset (1940-2023) on the S&P 500 and 3-month T-bill rates. 

In [2]:
import scipy as sp
from scipy.stats import norm
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import statsmodels.formula.api as sm
import datetime as dt
import matplotlib.dates as mdates



In [3]:
d=pd.read_excel("/Users/ouyangyingrun/Desktop/Shiller Long History June 2023.xlsx", sheet_name='Dataset', engine='openpyxl')
d['Date'] = pd.to_datetime(d['Date'])
d.columns = ['Date','SPXLev','N3m','DivYld','RealDiv']

d=d.sort_values('Date')
d=d.set_index('Date')

# Calculate the return index and 3-month T-Bill yield as a percentage
d['retidx']=d.SPXLev/d.SPXLev.iloc[0]
d['N3m']=d['N3m']/100

# Define the horizons in terms of months (1 year = 12 months, etc.)
k=[12,24,36,60,84,120]

# Calculate log excess returns for each horizon
for i in k:
 d['ret' + str(i) + 'm'] = d.retidx / d.retidx.shift(i) - 1 
 d['lret' + str(i) + 'm'] = np.log(d.retidx / d.retidx.shift(i))

 d['N3m' + str(i)] = d.N3m.rolling(i).apply(lambda x:np.sum(np.log(1 + x / 12)))

 d['lretex' + str(i) + 'm'] = d['lret' + str(i) + 'm'] - d['N3m' +str(i)] #realized log excess return

 d['lagdp' + str(i)] = d.DivYld.shift(i) #Lagging Dividend Yield over Horizon i
 d['rdivg' + str(i)] = np.log(d.RealDiv / d.RealDiv.shift(i)) #log real growth rate

d = d.dropna(axis=0, subset=["ret120m"])

pst_war = 1
if pst_war == 1:
 d = d.loc[d.index.year>1947,:]

In [5]:
# Calculate variance ratio for each horizon
results = {}
for i in k:
    # Calculate the variance of log excess returns for 1-year (12-month) and the current horizon
    var_1y = d['lretex12m'].var()
    var_i = d[f'lretex{i}m'].var()
    
    # Compute variance ratio
    VR_i = var_i / (i / 12 * var_1y)
    results[i] = VR_i

# Print results
print("Variance Ratio Test Results:")
for i, VR in results.items():
    print(f"Horizon: {i // 12} years, Variance Ratio: {VR:.4f}")

Variance Ratio Test Results:
Horizon: 1 years, Variance Ratio: 1.0000
Horizon: 2 years, Variance Ratio: 0.9771
Horizon: 3 years, Variance Ratio: 0.9132
Horizon: 5 years, Variance Ratio: 0.9498
Horizon: 7 years, Variance Ratio: 0.9741
Horizon: 10 years, Variance Ratio: 1.1471


-	Horizon: 1 Year, VR = 1.0000: A VR close to 1 suggest that returns are consistent with a random walk, or we say returns have zero autocorrelations.
-	Horizon: 2 Years, VR = 0.9771, Horizon: 3 Years, VR = 0.9132, Horizon: 5 Years, VR = 0.9498 and Horizon: 7 Years, VR = 0.9741: A VR less than 1 suggests returns have negative auto-correlation and exhibit mean reversion, prices tend to return to a historical mean over time. 
-	Horizon: 10 Years, VR = 1.1471: A VR greater than 1 suggest positive serial correlation, indicates a potential trend following behavior rather than mean reversion. 


In [6]:
# Define the results from Problem 1
variance_ratios = {
    1: 1.0000,
    2: 0.9771,
    3: 0.9132,
    5: 0.9498,
    7: 0.9741,
    10: 1.1471
}

# Define the parameters
T = len(d)  # Length of the data after filtering (post-1947)
significance_level = 0.02
critical_value = 2.326  # 2% significance level for two-tailed test

# Calculate the test statistics and determine if we reject the null hypothesis
for k, VR_i in variance_ratios.items():
    i = k
    J_r_i = VR_i - 1
    Z_i = (np.sqrt(T * i) * J_r_i) / np.sqrt(2 * (i - 1))
    
    # Determine if we reject the null hypothesis
    reject_null = abs(Z_i) > critical_value
    
    # Display results
    print(f"Horizon: {i} years, Variance Ratio: {VR_i:.4f}, Test Statistic Z_q: {Z_i:.4f}, Reject Null Hypothesis: {reject_null}")

Horizon: 1 years, Variance Ratio: 1.0000, Test Statistic Z_q: nan, Reject Null Hypothesis: False
Horizon: 2 years, Variance Ratio: 0.9771, Test Statistic Z_q: -0.6801, Reject Null Hypothesis: False
Horizon: 3 years, Variance Ratio: 0.9132, Test Statistic Z_q: -2.2325, Reject Null Hypothesis: False
Horizon: 5 years, Variance Ratio: 0.9498, Test Statistic Z_q: -1.1786, Reject Null Hypothesis: False
Horizon: 7 years, Variance Ratio: 0.9741, Test Statistic Z_q: -0.5875, Reject Null Hypothesis: False
Horizon: 10 years, Variance Ratio: 1.1471, Test Statistic Z_q: 3.2562, Reject Null Hypothesis: True


/var/folders/13/gb1xxdtj681bc6s351fs44n40000gn/T/ipykernel_98624/3175367419.py:20: RuntimeWarning: invalid value encountered in scalar divide
  Z_i = (np.sqrt(T * i) * J_r_i) / np.sqrt(2 * (i - 1))


For Horizon 1, 2, 3, 5, 7, Reject Null Hypothesis is False, indicating the hypothesis that log excess returns are serially uncorrelated and homoscedastic cannot be rejected at 2% significance level.
For Horizon 10, Reject Null Hypothesis is True, therefore the result is statistically significant and indicating evidence against the random walk hypothesis.


In [8]:
# Define horizons in terms of months
k = [12, 24, 36, 60, 84, 120]

# Loop through each horizon and print full regression summary
for i in k:
    #print(f"\nReturn Predictability Regression for {i//12} Year Horizon")
    
    # Define dependent and independent variables for each horizon:
    # Excess return predictability: y = lretex, x = lagdp
    y_return = d[f'lretex{i}m'].dropna()/(i//12)  # Log excess return series
    x_return = d[f'lagdp{i}'].loc[y_return.index]  # Lagged dividend yield

    # Run the regression for return predictability
    X_return = sm.add_constant(x_return)
    model_return = sm.OLS(y_return, X_return).fit(cov_type='HAC', cov_kwds={'maxlags': 1})
    #print(model_return.summary())  # Display full regression summary
    results_return_predictability[i] = {
        'Intercept': model_return.params['const'],
        'Beta': model_return.params[f'lagdp{i}'],
        'R2': model_return.rsquared,
        't-stat': model_return.tvalues[f'lagdp{i}']
    }

    #print(f"\nDividend Growth Predictability Regression for {i//12} Year Horizon")
    
    # Dividend growth predictability: y = rdivg, x = lagdp
    y_div_growth = d[f'rdivg{i}'].dropna()  # Log real dividend growth rate
    x_div_growth = d[f'lagdp{i}'].loc[y_div_growth.index]  # Lagged dividend yield

    # Run the regression for dividend growth predictability
    X_div_growth = sm.add_constant(x_div_growth)
    model_div_growth = sm.OLS(y_div_growth, X_div_growth).fit(cov_type='HAC', cov_kwds={'maxlags': 1})
    results_dividend_growth_predictability[i] = {
        'Intercept': model_div_growth.params['const'],
        'Beta': model_div_growth.params[f'lagdp{i}'],
        'R2': model_div_growth.rsquared,
        't-stat': model_div_growth.tvalues[f'lagdp{i}']
    }
    #print(model_div_growth.summary())  # Display full regression summary

# Display the results
print("Return Predictability Regression Results:")
for i, res in results_return_predictability.items():
    print(f"Horizon: {i//12} years, Intercept: {res['Intercept']:.4f}, Beta: {res['Beta']:.4f}, R2: {res['R2']:.4f}, t-stat: {res['t-stat']:.4f}")

print("\nDividend Growth Predictability Regression Results:")
for i, res in results_dividend_growth_predictability.items():
    print(f"Horizon: {i//12} years, Intercept: {res['Intercept']:.4f}, Beta: {res['Beta']:.4f}, R2: {res['R2']:.4f}, t-stat: {res['t-stat']:.4f}")

Return Predictability Regression Results:
Horizon: 1 years, Intercept: -0.0280, Beta: 3.1124, R2: 0.0677, t-stat: 6.0459
Horizon: 2 years, Intercept: -0.0125, Beta: 2.5962, R2: 0.0975, t-stat: 6.6803
Horizon: 3 years, Intercept: -0.0047, Beta: 2.3116, R2: 0.1240, t-stat: 7.6813
Horizon: 5 years, Intercept: -0.0075, Beta: 2.3060, R2: 0.1948, t-stat: 11.4600
Horizon: 7 years, Intercept: -0.0051, Beta: 2.1752, R2: 0.2420, t-stat: 13.5588
Horizon: 10 years, Intercept: -0.0030, Beta: 2.0084, R2: 0.3070, t-stat: 17.0161

Dividend Growth Predictability Regression Results:
Horizon: 1 years, Intercept: 0.0328, Beta: -0.2986, R2: 0.0044, t-stat: -1.0337
Horizon: 2 years, Intercept: 0.0605, Beta: -0.3883, R2: 0.0024, t-stat: -0.7913
Horizon: 3 years, Intercept: 0.0830, Beta: -0.2771, R2: 0.0007, t-stat: -0.4747
Horizon: 5 years, Intercept: 0.1423, Beta: -0.6191, R2: 0.0023, t-stat: -1.0138
Horizon: 7 years, Intercept: 0.1879, Beta: -0.6100, R2: 0.0018, t-stat: -0.9220
Horizon: 10 years, Intercept

Return Predictability Regression Results Interpretation
-	Positive betas (e.g., 3.1124 for 1-year, 2.5962 for 2-year, etc.) suggest that a higher dividend-price ratio tends to predict higher future returns. This is consistent with the theory that a high dividend yield indicates undervaluation, which may lead to higher returns.
-	R-squared increases with the horizon length (from 6.77% for 1 year to 30.7% for 10 years). This suggests that the predictive power of the dividend yield for returns strengthens over longer horizons.
-	T-statistics are higher than 2 in absolute value indicating the coefficients are statistically significant. 

Dividend Growth Predictability Regression Results Interpretation
-	The beta values are negative across all horizons (e.g., -0.2986 for 1-year, -1.0127 for 10-year), indicating an inverse relationship between the dividend yield and future dividend growth. This means that higher dividend yields are associated with lower subsequent dividend growth.
-	The R-squared values are very low (all below 1%), suggesting that the dividend-price ratio does not meaningfully explain variations in future dividend growth.
-	T-statistics for beta are mostly small and insignificant (below 2 in absolute value), indicating that the beta coefficients are not statistically different from zero.

In lecture we discussed high prices today must reflect either lower future returns or higher dividend growth or some combination. From two regression outcomes, we can say future returns and price (dividend to price ratio) shows a stronger relationship. 

